In [7]:
### This is about simulating a great number of times the Velib trafic in Rouen
import Distributions
using Distributions
srand(123);

In [8]:
#General Data
T = 1000 #horizon
n = 5 #number of stations : 3-7
dist_matrix = [0 3 5 7 7;
               2 0 2 5 5;
               4 2 0 3 3;
               8 6 4 0 2;
               7 7 5 2 0]; #distance between stations
terminal_number = [24 20 20 15 20]; #number of terminals at each station
routing_matrix = [0 0.2 0.3 0.2 0.3;
                  0.2 0 0.3 0.2 0.3;
                  0.2 0.25 0 0.25 0.3;
                  0.15 0.2 0.3 0 0.35;
                  0.2 0.25 0.35 0.2 0]; #routing matrix ?
leaving_per_hour = [2.8 3.7 5.5 3.5 4.6]; #mean number of bikes leaving each station per hour
N_bikes = convert(Integer, 91); #total number of bikes
N_stations = convert(Integer, 5); #number of stations

In [9]:
#Initial data
X0 = [20 1 0 0 0;
     1 15 1 0 0;
     0 1 17 1 0;
     0 0 1 13 1;
     0 0 0 1 18];

In [68]:
function first_bike_to_leave(on_run)
    first = on_run[1, :]
    for i in range(1,size(on_run)[1])
        if on_run[i,3] < first[3]
            first = on_run[i,:];
        end
    end
    return first
end

function pop_custom(on_run, first_bike)
    i0 = 1;
    for i in range(1,size(on_run)[1])
        if on_run[i,:] == first_bike
            i0 = i;
        end
    end
    if i0 == 1
        A = on_run[2:end, :];
    elseif i0 == size(on_run)[1]
        A = on_run[1:i0-1, :];
    else
        A = vcat(on_run[1:i0-1, :], on_run[i0+1:end, :]);
    end
    return A;
end

pop_custom (generic function with 1 method)

In [72]:
function simulate(T)
    #define distributions
    exp = Exponential(2); #PARAM TO BE CHANGED
    #**route = [Categorical(routing_matrix[i]) for i in range(N_stations)];
    route = Categorical([0.2,0.2,0.2,0.2,0.2]); #TO BE CHANGED
    #**leaving = Categorical(leaving_per_hour);
    leaving = Categorical([0.2,0.2,0.2,0.2,0.2]); #TO BE CHANGED
    #**dist = [Exponential(dist_matrix[i, j]) for i in range(N_stations), j in range(N_stations)];
    dist = Exponential(10); #TO BE CHANGED
    #time
    t = 0;
    #state
    X = X0;
    #current bikes on the run with their expected arrival time
    #on_run_bike = [1 2 rand(dist[1,2], 1);
    #               2 1 rand(dist[2,1], 1);
    #               2 3 rand(dist[2,3], 1);
    #               3 2 rand(dist[3,2], 1);
    #               3 4 rand(dist[3,4], 1);
    #               4 3 rand(dist[4,3], 1);
    #               4 5 rand(dist[4,5], 1);
    #               5 4 rand(dist[5,4], 1)];
    on_run_bike = [1 2 rand(dist);
                   2 1 rand(dist);
                   2 3 rand(dist);
                   3 2 rand(dist);
                   3 4 rand(dist);
                   4 3 rand(dist);
                   4 5 rand(dist);
                   5 4 rand(dist)];
    while (t < T)
        #get expected stay time
        stay = rand(exp);
        #check if there is an arrival time of a bike on the run before stay time.
        first_bike = first_bike_to_leave(on_run_bike);
        #if the stay time is shorter than the first bike to arrive, it is a departure
        if first_bike[3] > t + stay
            station = rand(leaving); #station from which the bike leaves
            #**destination = rand(route[station]); #destination station according to the routing matrix
            destination = rand(route); #TO BE CHANGED
            X[station, destination] = X[station, destination] + 1;
            X[station, station] = X[station, station] - 1;
            #**on_run_bike.append([station destination (t+rand(dist[station, destination], 1))]);
            on_run_bike = [on_run_bike; [station destination t+stay+rand(dist)]]; #TO BE CHANGED
            t = t + stay;
            println("departure ", t);
        else
            station = convert(Integer, first_bike[2]);
            origin = convert(Integer, first_bike[1]);
            time = first_bike[3];
            X[origin, station] = X[origin, station] - 1;
            X[station, station] = X[station, station] + 1;
            on_run_bike = pop_custom(on_run_bike, first_bike);
            t = t + time;
            println("arrival ", t);
        end
    end
    result = 0;
    return result
end

simulate (generic function with 1 method)

In [73]:
simulate(100)

arrival 0.4265850817152866
arrival 0.9683318318906726
arrival 2.1737778144211752
arrival 5.024176979477375
arrival 8.916096217145762
arrival 14.24855381597553
arrival 19.907705191131036
departure 20.92800456371015
departure 22.932451779319674
departure 24.75282917388133
departure 26.911668854918524
departure 27.25421235785409
arrival 54.62989879709791
arrival 83.79046274917143
arrival 113.30958916793384


0